In [1]:
#Load Libraries
import io
from IPython.display import clear_output
import winsound
import json
import cv2
import requests
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime,timedelta
import time as time
import pandas as pd

def makeNoise(n=1):
#play beep sound
    frequency = 500  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    for i in range(n):
        winsound.Beep(frequency, duration)
def sleep(strlist,dur=1):
    bar_length = 50
    for i in range(dur):
        clear_output(wait = True)
        for string in strlist:
            print(string)
        block = int(round(bar_length * (i)/dur))
        print("Sleep time: [{0}] {1:.0f}/{2:.0f} mins".format( "#" * block + "-" * (bar_length - block), i, dur))
        time.sleep(60) # 60s = 1min
    clear_output(wait = True)
    print("Sleep time: [{0}] {1:.0f}/{2:.0f} mins".format( "#" * bar_length, dur, dur))
#create api-key list
apikeystore = [
    'K84672750088957','K87887256288957','K87576188288957','K84236823888957','K87600253188957','K84253289488957''K84124214688957',
    'K84672750088957','K87887256288957','K87576188288957','K84236823888957','K87600253188957','K84253289488957','K84124214688957',
    'K84672750088957','K87887256288957','K87576188288957','K84236823888957','K87600253188957','K84253289488957','K84124214688957',
    
    ]
        
#api function to convert jpg to text 
def ocrtotxt(arg):
    # arg=[fname,api_key]
    fname = arg[0]
    api_key = arg[1]
    
    #pre-process image
    try:
        img = cv2.imread(fname) 
        h,w,_ = img.shape
        roi = img[0:h,0:w,:]
        _, compressedimage = cv2.imencode(".jpg", roi, [1, 80])
    except:
        return ('jpg file error: ' + fname)
    
    # post img-Ocr request
    file_bytes = io.BytesIO(compressedimage)
    try:
        result = requests.post('https://api.ocr.space/parse/image',
                              files={fname: file_bytes},
                              data= {'apikey': api_key,
                                     'isTable': True
                                    }
                                  )
        return result.content.decode()
    except requests.exceptions.RequestException as e:
        return 'request error'
    

# employ worker to bulk extracting    
def convert_jpgs_to_txts(argumentlist):
#return list of string - Jsontype
    with ThreadPoolExecutor(max_workers=12) as pool:
        result = list(pool.map(ocrtotxt,argumentlist))
    return result

#write to Json-text file
def writeRawJson(results):
    #save txt
    fmap = []
    for output in results:
        if ('ParsedResults' in output[1]) and ('ParsedText' in output[1]):
            f = open(file=output[0], mode='w')
            f.write(output[1])
            f.close()
            fmap.append(output[0])
        else:
            fmap.append("Error")
    #create mapping file
    df.loc[df[df.jpg.isin(jpgnames)].index,'json'] = fmap

    #save mapping file
    #df.to_csv(json_folder + "/" + json_folder.split('/')[-1] + ' jsonfilemap.csv')
    df.to_csv(jpgmappingfname,index=False)
    
#write to Parsed-text file
def writeParseTxt(results):
    #save txt
    fmap = []
    for output in results:
        if ('ParsedResults' in output[1]) and ('ParsedText' in output[1]):
            content = json.loads(output[1]).get('ParsedResults')[0].get('ParsedText')
            f = open(file=output[0], mode='w')
            f.write(content)
            f.close()
            fmap.append(output[0])
        else:
            print("error on file: ", output[0] , ',' , output[1] )
            fmap.append("Error")
            
    #create mapping file
    df.loc[df[df.jpg.isin(jpgnames)].index,'txt'] = fmap

    #save mapping file
    #df.to_csv(txt_folder + "/" + json_folder.split('/')[-1] + ' txtfilemap.csv')
    df.to_csv(jpgmappingfname,index=False)

In [4]:
########### CONTROL BOARD ########################################
jpgmappingfname = 'D:/tmrisc/filemap.csv'                
json_folder     = 'D:/tmrisc/json/'
txt_folder      = 'D:/tmrisc/txt/'
########### INITATE ########################################
df = pd.read_csv(jpgmappingfname) # read file
if "json" not in df: df['json'] = "NAN" # checking available of collumns json
if "txt" not in df: df['txt'] = "NAN"  # checking available of collumns txt
#create api-key list
apikey_list=[]
maxallowdayreq = 500 # max 500 request/key/day
for key in apikeystore:
    apikey_list = apikey_list + ([key]*maxallowdayreq)
print('Remain {} files'.format(df.loc[df['txt'].isna(),'jpg'].count()))

Remain 0 files


In [3]:
# AUTO-RUN
# First SetUp
maxseq = int(input('Number of Rounds (hours):   '))
dayrequest = int(input('the last number of request today [0,18]: ')) # [0 to 18] is the last number of request in a day, 
waittime = int(input('the script to run after (mins):   ')) # minutes # setting time elapse to run
starttime = datetime.now()
sleep(['the converting to start at '+ (datetime.now()+timedelta(minutes=waittime)).strftime('%H:%M:%S')],dur=waittime) # minutes
print('Work start at: ', starttime.strftime('%H:%M:%S'))
automode = True
seq=0
totalfile = df.loc[df['txt'].isna(),'jpg'].count()
fileremain = totalfile
#check to re-run
while automode == True and seq<maxseq:  
    seq+=1
    dayrequest +=1
    ########################################## Reset ##########################################################################
    #create api&jpg list
    jpg_list=list(df.loc[df['txt'].isna(),'jpg']) # load jpg list
    end = min(len(jpg_list),180)# check to update start:end slicing
    apikeys = apikey_list[(dayrequest-1)*180:(dayrequest)*180]
    jpgnames = jpg_list[0:end] # update name lists
    jsonnames =  list(map(lambda jpgname: json_folder + "/" + jpgname.split('/')[-1].replace('jpg','txt'),jpgnames)) # update name lists
    txtnames = list(map(lambda jpgname: txt_folder + "/" + jpgname.split('/')[-1].replace('jpg','txt'),jpgnames))# update name lists
    api_argument = list(zip(jpgnames,apikeys)) #new list of arguments
    #clear_output(wait = True) # clear screen
    ########################################## Run ##########################################################################
    # Run API request
    print('coverting.....')
    results = convert_jpgs_to_txts(api_argument)
    # Write to file
    writeRawJson(list(zip(jsonnames,results)))
    writeParseTxt(list(zip(txtnames,results))) 
    #check to re-run
    fileremain = df.loc[df['txt'].isna(),'jpg'].count()
    if fileremain == 0 or seq==maxseq                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    :
        automode = False # end auto-mode
    else:
        makeNoise(2) #raise alert
        strlist = ['Work start at \n'+ starttime.strftime('%H:%M:%S'),
                    'Total {0}/{1} files were proceed after seq.{2}/{3}'.format(totalfile - fileremain,totalfile,seq,maxseq),
                  'We need sleeping about 60 minutes from ' + datetime.now().strftime('%H:%M:%S')]
        #sleep(strlist,60) # wait 60 mins as API's require
        print('Total {0}/{1} files were proceed after seq.{2}/{3}'.format(totalfile - fileremain,totalfile,seq,maxseq))
        print('We need sleeping about 60 minutes from ' + datetime.now().strftime('%H:%M:%S'))
        time.sleep(3600) # wait 60 mins as API's require 
    
finishtime = datetime.now()
#clear_output(wait = True)
print('Work start  at {0} \n     finish at {1} '.format(starttime.strftime('%H:%M:%S'),finishtime.strftime('%H:%M:%S')))
print('Total {0}/{1} files were proceed after seq.{2}/{3}'.format(totalfile - fileremain,totalfile,seq,maxseq))
print('dayrequest = ',dayrequest)
makeNoise(5) #raise alert to stop

Sleep time: [##################################################] 0/0 mins
Work start at:  14:42:20
coverting.....
Total 180/490 files were proceed after seq.1/3
We need sleeping about 60 minutes from 14:44:56
coverting.....
Total 360/490 files were proceed after seq.2/3
We need sleeping about 60 minutes from 15:47:40
coverting.....
Work start  at 14:42:20 
     finish at 16:49:46 
Total 490/490 files were proceed after seq.3/3
dayrequest =  16


last dayrequest =  13

In [ ]:
print('the last coverting is at',finishtime.strftime('%H:%M:%S'))
print('it is {:.0f} mins from the last sending request'.format((datetime.now()-finishtime).seconds/60))
print('dayrequest = ',dayrequest)

In [7]:
df[df.output.isna()].count()

Unnamed: 0       1037
pdf              1037
jpg              1037
json             1037
txt              1037
output              0
request_no          0
date               15
thaodien           15
phuoclong          15
vanthanh           15
rachchiec          15
suoitien           15
daihocquocgia      15
hightech           15
thuduc             15
binhthai           15
anphu              15
tancang            15
depot              15
factory            15
brick              15
plastering         15
metalcladding      15
ceiling            15
epoxy              15
painting           15
stone              15
partition          15
waterproofing      15
topping            15
dtype: int64

# SEMI-AUTO RUN

automode = False
jpg_list = list(df.jpg)
# Check Setting 
if automode: 
    start_index = end_index + 1 # auto re-Indexing
    end_index = start_index + 180 - 1 # auto re-Indexing
    waittime = int(input('the script to run after (mins):   ')) # minutes # setting time elapse to run
else:
    start_index = int(input('Start from index:   ')) # manual input
    end_index = int(input('End at index:   ')) # manual input
    waittime = int(input('the script to run after (mins):   ')) # minutes # setting time elapse to run

########################################## Reset ##########################################################################
sleep(['the script to run after {:.0f} minutes'.format(waittime*60)],waittime) # Seconds
end = min(len(jpg_list),end_index+1)# check to update start:end slicing
apikeys = apikey_list[start_index:end]
jpgnames = jpg_list[start_index:end] # update name lists
jsonnames =  list(map(lambda jpgname: json_folder + "/" + jpgname.split('/')[-1].replace('jpg','txt'),jpgnames)) # update name lists
txtnames = list(map(lambda jpgname: txt_folder + "/" + jpgname.split('/')[-1].replace('jpg','txt'),jpgnames))# update name lists
api_argument = list(zip(jpgnames,apikeys)) #new list of arguments
print('total {0} files [{1}:{2}] to be converted'.format(len(api_argument),start_index,end_index)) #report
########################################## Run ##########################################################################
# Run API request
print('start coverting  [{}:{}] at'.format(start_index,end_index),datetime.now().strftime('%H:%M:%S'))
results = convert_jpgs_to_txts(api_argument)
# Write to file
writeRawJson(list(zip(jsonnames,results)))
writeParseTxt(list(zip(txtnames,results)))
# report
finishtime = datetime.now()
print('finish coverting [{}:{}] at'.format(start_index,end_index),finishtime.strftime('%H:%M:%S'))
#check to re-run
if end == len(jpg_list):
    makeNoise(1) #raise alert
    automode = False # end auto-mode

# Report the last converting
automode = False
print('the last coverting is [{}:{}] at'.format(start_index,end_index),finishtime.strftime('%H:%M:%S'))
print('it is {:.0f} mins from the last sending request'.format((datetime.now()-finishtime).seconds/60))